In [61]:
from elasticsearch import Elasticsearch, helpers
from rank_bm25 import BM25Okapi
import pandas as pd
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import re
from tqdm import tqdm
import pickle
import numpy as np

In [62]:
wiki_dataset = pd.read_json('/opt/ml/data/wikipedia_documents.json',orient='index')
train_dataset = load_from_disk('/opt/ml/data/train_dataset/train').to_pandas()
valid_dataset = load_from_disk('/opt/ml/data/train_dataset/validation').to_pandas()


In [63]:
def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\\n', ' ', text) # remove newline character
    text = re.sub(r'\s+', ' ', text) # remove continuous spaces
    text = re.sub(r'#', ' ', text)

    return text

In [64]:
wiki_dataset['text'] = wiki_dataset['text'].apply(preprocess)
wiki_dataset = wiki_dataset.drop_duplicates(['text','title'],ignore_index=True) # 중복 제거 후 인덱스 재설정
#train_dataset['context'] = train_dataset['context'].apply(preprocess)
#valid_dataset['context'] = valid_dataset['context'].apply(preprocess)

In [65]:
wiki_corpus = list(set([example_wiki for example_wiki in wiki_dataset['text']]))
train_context = train_dataset['context']
valid_context = valid_dataset['context']
train_query = train_dataset['question']
valid_query = valid_dataset['question']

In [66]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
tokenized_corpus = [tokenizer.tokenize(context) for context in wiki_corpus]
bm25 = BM25Okapi(tokenized_corpus)

INDEX_NAME = "wiki_index"
INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],
          }
        }
      }
    }
  },
  "mappings": {
      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }
  }
}

Token indices sequence length is longer than the specified maximum sequence length for this model (619 > 512). Running this sequence through the model will result in indexing errors


In [67]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : wiki_dataset.iloc[i]['document_id'],
        'title' : wiki_dataset.iloc[i]['title'],
        'content' : wiki_dataset.iloc[i]['text']
    }
    for i in range(wiki_dataset.shape[0])
]

In [68]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [69]:
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '89aa4310917b',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'tWsEBuE7ReuuGa-pZkywtw',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [70]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-70-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-70-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_index'}

In [71]:
import time
start_time = time.time()
try:
    response = helpers.bulk(es, docs)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)
    
end_time = time.time()

print(end_time-start_time)


RESPONSE: (56044, [])
100.95854902267456


In [72]:
try:
    res = es.search(index=INDEX_NAME, q=train_query[0], size=100)
except:
    res = es.search(index=INDEX_NAME, q=train_query[0], size=100)

In [73]:
print(train_query[0])
res['hits']['hits']

대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?


[{'_index': 'wiki_index',
  '_type': '_doc',
  '_id': '18293',
  '_score': 36.58704,
  '_source': {'title': '미국 상원',
   'content': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05'}},
 {'_ind

In [74]:
# reader_data

train_data_with_elastic = {}
train_data_with_elastic_score = {}
error_questions = []
top_k_num = 200
for i in tqdm(range(len(train_query))):
    try:
        res = es.search(index=INDEX_NAME, q=train_query[i], size=top_k_num)
        hits = res['hits']['hits']

        top_k_list = []
        top_k_score = []
        for hit in hits:
            ctx = hit['_source']['content']
            score = hit['_score']
            top_k_list.append(ctx)
            top_k_score.append(score)
        train_data_with_elastic[train_query[i]] = top_k_list
        train_data_with_elastic_score[train_query[i]] = top_k_score
    except Exception as e:
        error_questions.append([e, train_query[i]])
        train_data_with_elastic[train_query[i]] = bm25.get_top_n(tokenizer.tokenize(train_query[i]), wiki_corpus, n=top_k_num)
        temp_score = bm25.get_scores(tokenizer.tokenize(train_query[i])).tolist()
        temp_score.sort(reverse=True)
        train_data_with_elastic_score[train_query[i]] = temp_score[:top_k_num]
    
    

100%|██████████| 3952/3952 [05:41<00:00, 11.58it/s]


In [75]:
# reader_data
valid_data_with_elastic = {}
valid_data_with_elastic_score = {}
error_valid_questions = []
top_k_num = 200
for i in tqdm(range(len(valid_query))):
    try:
        res = es.search(index=INDEX_NAME, q=valid_query[i], size=top_k_num)
        hits = res['hits']['hits']

        top_k_list = []
        top_k_score = []
        for hit in hits:
            ctx = hit['_source']['content']
            score = hit['_score']
            top_k_list.append(ctx)
            top_k_score.append(score)
        valid_data_with_elastic[valid_query[i]] = top_k_list
        valid_data_with_elastic_score[valid_query[i]] = top_k_score
    except Exception as e:
        error_valid_questions.append([e, valid_query[i]])
        valid_data_with_elastic[valid_query[i]] = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        temp_score = bm25.get_scores(tokenizer.tokenize(valid_query[i])).tolist()
        temp_score.sort(reverse=True)
        valid_data_with_elastic_score[valid_query[i]] = temp_score[:top_k_num]
    
    

100%|██████████| 240/240 [00:22<00:00, 10.77it/s]


In [76]:
with open('/opt/ml/data/elastic_new_train_500.bin', "wb") as file:
    pickle.dump(train_data_with_elastic, file)
with open('/opt/ml/data/elastic_new_train_score_500.bin', "wb") as file:
    pickle.dump(train_data_with_elastic_score, file)
with open('/opt/ml/data/elastic_new_valid_500.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic, file)
with open('/opt/ml/data/elastic_new_valid_score_500.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic_score, file)

In [77]:
test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
test_query = test_dataset['question']

In [78]:
test_data_with_elastic = {}
test_data_with_elastic_score = {}
error_test_questions = []
top_k_num = 200
for i in tqdm(range(len(test_query))):
    try:
        res = es.search(index=INDEX_NAME, q=test_query[i], size=top_k_num)
        hits = res['hits']['hits']
        top_k_list = []
        top_k_score = []
        for hit in hits:
            ctx = hit['_source']['content']
            score = hit['_score']
            top_k_list.append(ctx)
            top_k_score.append(score)
        test_data_with_elastic[test_query[i]] = top_k_list
        test_data_with_elastic_score[test_query[i]] = top_k_score
    except Exception as e:
        error_test_questions.append([e, test_query[i]])
        test_data_with_elastic[test_query[i]] = bm25.get_top_n(tokenizer.tokenize(test_query[i]), wiki_corpus, n=top_k_num)
        temp_score = bm25.get_scores(tokenizer.tokenize(test_query[i])).tolist()
        temp_score.sort(reverse=True)
        test_data_with_elastic_score[test_query[i]] = temp_score[:top_k_num]

100%|██████████| 600/600 [00:51<00:00, 11.75it/s]


In [79]:
with open('/opt/ml/data/elastic_retrieval_500.bin', "wb") as file:
    pickle.dump(test_data_with_elastic, file)
with open('/opt/ml/data/elastic_retrieval_score_500.bin', "wb") as file:
    pickle.dump(test_data_with_elastic_score, file)

In [80]:
# with open('/opt/ml/data/elastic_train_100.bin', "rb") as file:
#     train_data_with_elastic = pickle.load(file)
with open('/opt/ml/data/elastic_new_valid_500.bin', "rb") as file:
    valid_data_with_elastic = pickle.load(file)

In [86]:
for i in range(len(valid_query)):
    print(valid_query[i])
    print(valid_data_with_elastic[valid_query[i]][3])
    break

처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?
남조선해방전략당 사건은 박정희 정부때인 1968년 정보기관인 중앙정보부가 통일혁명당 사건을 수사하면서 1968년 7월 30일에 권재혁 등 13명을 강제 연행해 3∼53일간 불법구금하고 고문과 가혹행위로 허위 진술을 강요하여 이들의 친목모임에 대해 남조선해방전략당이라고 하면서 8월 24일 '간첩사건'이라고 발표한 사건이다. 연행된 사람들은 국가보안법 위반과 반공법 위반, 내란음모 등의 혐의로 기소되어 징역 7년에서 사형을 각각 선고받았다. 반국가단체 구성 및 수괴죄, 내란예비음모죄, 간첩죄가 적용돼 1969년 9월 대법원에서 사형이 확정된 권재혁은 두 달 만인 11월 4일 교수형 집행됐고 이일재는 무기징역을 선고받은 뒤 1988년 8·15 특별사면으로 석방, 이강복은 징역 10년형을 선고받고 대전교도소에 복역 중 1971년 10월 암으로 옥중에서 사망했다. 이후 시민사회에서 진상규명을 요구하는 목소리가 계속되었고 진실과 화해를 위한 과거사정리위원회는 2009년 10월 12일에 "남조선해방전략당 사건은 1968년 중앙정보부가 권재혁 등 13명을 강제 연행한 뒤 불법구금과 가혹행위를 통해 조작한 사건이다"고 하면서 국가에 피해자 보상과 재심을 권고했다. 서울고등법원 형사3부(부장판사 강영수)는 남조선해방 전략당 사건으로 징역 3년 자격정지 3년 집행유예 5년을 선고받은 고인이 된 나경일의 부인 임모(84)씨 등 유가족 5명이 낸 재심에서 나경일에 대해 무죄를 선고했다. 대법원 3부(주심 박보영 대법관)는 내란예비음모와 간첩 등의 혐의로 기소돼 사형을 선고받은 권재혁의 유족이 청구한 재심에서 권재혁에게 무죄를 선고했다. 서울중앙지법 민사합의10부(부장 이은희)는 "남조선해방전략당 사건의 권재혁 유족에게 35억원, 7년~무기징역 형을 받은 이일재 등 3명의 유족들에게 8억~14억여원씩을 지급하라"고 했다. 문재인 정부가 출범한 이후 문재인 대통령이 임명한 문무일 검찰총장은 이 사건에서 재심을 청구하지 않은 피고인에

In [87]:
res = es.search(index=INDEX_NAME, q=valid_query[0], size=100)

In [88]:
res

{'took': 48,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 26.632221,
  'hits': [{'_index': 'wiki_index',
    '_type': '_doc',
    '_id': '17913',
    '_score': 26.632221,
    '_source': {'title': '기업 경영에 대한 법규',
     'content': "기업을 운영하는 데 있어서는 우선 인적 주체가 누구이냐가 문제되는바, 이는 기업의 형태 문제와 관련된다. 상법과 민법은 개인기업의 형태를 택할 때의 '상인'에 관한 개념과, 공동기업의 형태를 택할 때의 '영업자'와 '익명조합원' 또는 '공동사업자'에 관한 개념과, 법인기업의 형태를 택할 때의 '법인'의 개념을 규제한다. 그리고 법인기업(회사기업)의 설립에 관하여는 정관(定款)의 작성과 설립등기(設立登記)에 관하여 자세한 규제를 하고 있다. 회사라고 하더라도 주식회사(株式會社)이냐, 유한회사(有限會社)이냐, 합명회사(合名會社)이냐 혹은 합자회사(合資會社)이냐에 따라서 각각 다른 내용의 규제를 하고 있다. 기업의 규모에 관하여도 법적 규제가 있다. 예컨대 유한회사에 있어서는 자본금을 천만원 이상으로 규제하고 (상법 546조 1항), 금융기관의 자본금은 250억원 이상으로 규제한다(은행법 16조 1항). 이와 같은 예는 증권거래법에도 있고, 기타의 특별법에도 산재한다. 일단 설립된 주식회사가 증자(增資)를 통하여 규모를 확대하는 경우와 감자(減資)를 통하여 규모를 축소시키려는 경우에 상법 기타 특별법의 규제가 가해진다. 기업이 조직변형을 하는 경우에도 법적 규제를 받아야 한다. 이를테면 주식회사는 유한회사로, 그리고 유한회사는 주식회사로 조직변경할 수 있다.

In [60]:
temp_df = wiki_dataset.drop_duplicates(['text','title'])
temp_df[temp_df['title'] == '전효숙']

,text,corpus_source,url,domain,title,author,html,document_id
4473,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",위키피디아,TODO,NaN,전효숙,NaN,NaN,4473
4474,2006년 8월 16일 노무현 대통령으로부터 헌법재판소장으로 지명되어 여성 최초의 ...,위키피디아,TODO,NaN,전효숙,NaN,NaN,4474
